In [78]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip
from glob import glob
from astropy.table import Table
import os
import re
from astropy.visualization import MinMaxInterval, ZScaleInterval, LinearStretch, LogStretch, SqrtStretch, ImageNormalize
#from astroscrappy import detect_cosmics
from glob import glob
import warnings

In [26]:
log = Table.read('/Volumes/obs-data/obs/2024feb11/kred_test/KCWI_20240211.log', format = 'csv')
kcrm = log[log['Camera'] == 'RED']

kcrm.show_in_notebook()

idx,No.,Camera,Object,ImType,GroupID,State Name,Target,LST,UT,RA,DEC,AIRMASS,Exptime,Binning,Ampmode,CCD Speed,CCD Mode,Grating,Filter,Central wave,NS mask,IFU,Rot Mode,Sky PA
0,kr240211_00001.fits,RED,TEST,DARK,2024-02-11-1,undefined,unknown,01:02:54.62,02:02:13.20,22:46:55.10,+00:04:51.6,13.37,1.0,"2,2",L2U2,0,0,None,--,0.0,Open,Medium,stationary,0.0
1,kr240211_00002.fits,RED,focus,FOCUS,2024-02-11-2,kcwi_focus,unknown,01:41:09.94,02:40:22.25,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
2,kr240211_00003.fits,RED,focus,FOCUS,2024-02-11-3,kcwi_focus,unknown,01:42:33.82,02:41:45.90,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
3,kr240211_00004.fits,RED,focus,FOCUS,2024-02-11-4,kcwi_focus,unknown,01:43:57.70,02:43:09.55,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
4,kr240211_00005.fits,RED,focus,FOCUS,2024-02-11-5,kcwi_focus,unknown,01:45:19.52,02:44:31.15,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
5,kr240211_00006.fits,RED,focus,FOCUS,2024-02-11-6,kcwi_focus,unknown,01:46:40.32,02:45:51.73,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
6,kr240211_00007.fits,RED,focus,FOCUS,2024-02-11-7,kcwi_focus,unknown,01:48:01.13,02:47:12.32,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
7,kr240211_00008.fits,RED,focus,FOCUS,2024-02-11-8,kcwi_focus,unknown,01:49:22.96,02:48:33.93,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
8,kr240211_00009.fits,RED,focus,FOCUS,2024-02-11-9,kcwi_focus,unknown,01:50:46.83,02:49:57.57,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0
9,kr240211_00010.fits,RED,focus,FOCUS,2024-02-11-10,kcwi_focus,unknown,01:52:10.71,02:51:21.22,22:46:55.10,+00:04:51.6,13.37,7.0,"2,2",L2U2,1,1,RM2,--,7799.9746094,Open,Medium,stationary,0.0


In [18]:
sci_frames = np.arange(80, 198)
# badframe = [127, 128, 129, 130]
base = '/Volumes/obs-data/obs/2024feb11/kred_test/'
redux = os.path.join(base, 'redux')
# sci_frames = np.arange(117, 120)

In [62]:
# group frames based on PA
continuous = True

obj_dict = {}

for i, row in enumerate(kcrm):

    if not os.path.exists(os.path.join(base, row['No.'])):
        print(os.path.join(base, row['No.']))
        continue

    if row['CAMERA'] != 'RED':
        continue

    if row['ImType'] != 'OBJECT':
        continue

    key = '{0}_{1:d}_{2}'.format(row['Object'], int(np.round(row['Sky PA'])), row['State Name'])

    if key not in obj_dict.keys():
        obj_dict[key] = []
    obj_dict[key].append(row['No.'])


if continuous:  # requires file numbers to be continuous
    obj_dict_cont = {}
    for key in obj_dict.keys():
        fns = obj_dict[key]
        fnos = [int(fn.split('_')[1][:5]) for fn in obj_dict[key]]

        if len(fnos) <= 1:
            continue

        results_no = []
        tmp_no = [fnos[0]]
        results_fn = []
        tmp_fn = [fns[0]]
        for i in range(1, len(fnos)):
            if fnos[i] == fnos[i - 1] + 1:
                tmp_no.append(fnos[i])
                tmp_fn.append(fns[i])
            else:
                results_no.append(tmp_no)
                results_fn.append(tmp_fn)
                tmp_no = [fnos[i]]
                tmp_fn = [fns[i]]
        results_no.append(tmp_no)
        results_fn.append(tmp_fn)

        if len(results_no) == 1:
            # do nothing
            obj_dict_cont[key] = obj_dict[key]
        else:
            for i, result_fn in enumerate(results_fn):
                obj_dict_cont[key+'_{0:d}'.format(i)] = result_fn

    obj_dict = obj_dict_cont

obj_dict


{'desi094_0_arcs': ['kr240211_00080.fits',
  'kr240211_00081.fits',
  'kr240211_00082.fits',
  'kr240211_00083.fits',
  'kr240211_00084.fits',
  'kr240211_00085.fits',
  'kr240211_00086.fits',
  'kr240211_00087.fits'],
 'desi094-sky_0_arcs': ['kr240211_00088.fits', 'kr240211_00089.fits'],
 'desi094_90_arcs': ['kr240211_00090.fits',
  'kr240211_00091.fits',
  'kr240211_00092.fits',
  'kr240211_00093.fits'],
 'he3_90_arcs': ['kr240211_00094.fits',
  'kr240211_00095.fits',
  'kr240211_00096.fits',
  'kr240211_00097.fits',
  'kr240211_00098.fits'],
 'he3_90_H2-long': ['kr240211_00099.fits',
  'kr240211_00100.fits',
  'kr240211_00101.fits',
  'kr240211_00102.fits'],
 'he3_90_H2-comb-large': ['kr240211_00103.fits',
  'kr240211_00104.fits',
  'kr240211_00105.fits'],
 'macs0744_0_arcs': ['kr240211_00106.fits',
  'kr240211_00107.fits',
  'kr240211_00108.fits',
  'kr240211_00109.fits',
  'kr240211_00110.fits',
  'kr240211_00111.fits',
  'kr240211_00112.fits',
  'kr240211_00113.fits'],
 'macs0744

In [63]:
import json

obj_dict = json.load(open('/Volumes/obs-data/obs/2024feb11/kred_test/kcrm_group.json', 'r'))

obj_dict

{'desi094_0_arcs_300': ['kr240211_00080.fits',
  'kr240211_00081.fits',
  'kr240211_00082.fits',
  'kr240211_00083.fits',
  'kr240211_00084.fits',
  'kr240211_00085.fits',
  'kr240211_00086.fits',
  'kr240211_00087.fits'],
 'desi094-sky_0_arcs_300': ['kr240211_00088.fits', 'kr240211_00089.fits'],
 'desi094_90_arcs_300': ['kr240211_00090.fits',
  'kr240211_00091.fits',
  'kr240211_00092.fits',
  'kr240211_00093.fits'],
 'he3_90_arcs_45': ['kr240211_00096.fits',
  'kr240211_00097.fits',
  'kr240211_00098.fits'],
 'he3_90_H2-long_60': ['kr240211_00100.fits',
  'kr240211_00101.fits',
  'kr240211_00102.fits'],
 'he3_90_H2-comb-large_60': ['kr240211_00103.fits',
  'kr240211_00104.fits',
  'kr240211_00105.fits'],
 'macs0744_0_arcs_300': ['kr240211_00106.fits',
  'kr240211_00107.fits',
  'kr240211_00108.fits',
  'kr240211_00109.fits',
  'kr240211_00110.fits',
  'kr240211_00111.fits',
  'kr240211_00112.fits',
  'kr240211_00113.fits'],
 'macs0744_90_arcs_300': ['kr240211_00114.fits',
  'kr240211

In [15]:
wavemap_path = glob(os.path.join(redux, '*wavemap.fits'))[0]
#wavemap = fits.getdata(wavemap_path)
wavemap_path

'/Volumes/obs-data/obs/2024feb11/kred_test/redux/kr240211_00024_wavemap.fits'

In [64]:
def read_proctab(tfil='kcwi.proc'):
    # from official DRP
    proctab = Table.read(tfil, format='ascii.fixed_width')
    proctab.dtypes = ('int32', 'S24', 'int64', 'S9', 'S12',
                            'float64', 'S4', 'S6', 'S5', 'float64',
                            'float64', 'S4', 'S5', 'float64', 'int32',
                            'S5', 'S25', 'S25', 'S25')

    # format columns
    proctab['GANG'].format = '7.2f'
    proctab['CWAVE'].format = '8.2f'
    proctab['MJD'].format = '15.6f'
    # prevent string column truncation
    for col in proctab.itercols():
        if col.dtype.kind in 'SU':
            proctab.replace_column(col.name, col.astype('object'))

    return proctab

def search_proctab(proctab, frame, target_type=None, target_group=None,
        nearest=False):
    # copied from official DRP

    if target_type is not None and proctab is not None:
        tab = proctab[(proctab['CAM'] ==
                            frame.header['CAMERA'].upper().strip())]
        # get target type images
        tab = tab[(proctab['TYPE'] == target_type)]
        # BIASES must have the same CCDCFG
        if 'BIAS' in target_type:
            tab = tab[(tab['DID'] == int(frame.header['CCDCFG']))]
            if target_group is not None:
                tab = tab[(tab['GRPID'] == target_group)]
        # raw DARKS must have the same CCDCFG and TTIME
        elif target_type == 'DARK':
            tab = tab[tab['DID'] == int(frame.header['CCDCFG'])]
            tab = tab[tab['TTIME'] == float(frame.header['TTIME'])]
            if target_group is not None:
                tab = tab[tab['GRPID'] == target_group]
        # MDARKS must have the same CCDCFG, will be scaled to match TTIME
        elif target_type == 'MDARK':
            tab = tab[(tab['DID'] == int(frame.header['CCDCFG']))]
        elif target_type == 'OBJECT':
            tab = tab[tab['GRPID'] == target_group]
        else:
            tab = tab[(tab['CID'] == frame.header['STATEID'])]
        # Check if nearest entry is requested
        if nearest and len(tab) > 1:
            tfno = frame.header['MJD']
            minoff = 99999
            trow = None
            for row in tab:
                off = abs(row['MJD'] - tfno)
                if off < minoff:
                    minoff = off
                    trow = row
            if trow is not None:
                tab = tab[(tab['MJD'] == trow['MJD'])]
    else:
        tab = None
    return tab

In [82]:
obj_dict = json.load(open('/Volumes/obs-data/obs/2024feb11/kred_test/kcrm_group.json', 'r'))
base = '/Volumes/obs-data/obs/2024feb11/kred_test/'
redux = os.path.join(base, 'redux')

if not os.path.isdir(os.path.join(redux, 'int_median')):
    os.mkdir(os.path.join(redux, 'int_median'))

for obj in list(obj_dict.keys()):

    if len(obj_dict[obj]) >= 3:
        msci = np.median([fits.getdata(os.path.join(redux, re.sub('.fits', '_int.fits', f))) for f in obj_dict[obj]], axis = 0)
    else:
        msci = np.min([fits.getdata(os.path.join(redux, re.sub('.fits', '_int.fits', f))) for f in obj_dict[obj]], axis = 0)
    mhdu = fits.open(os.path.join(redux, re.sub('.fits', '_int.fits', obj_dict[obj][0])))
    mhdu[0].data = msci
    mhdu.writeto(os.path.join(redux, 'int_median', '%s_int.fits'%obj), overwrite = True)
    print('######################%s##############'%obj)
    for f in obj_dict[obj]:
        print(f)
        img_hdu = fits.open(os.path.join(redux, re.sub('.fits', '_int.fits', f)))
        img = img_hdu[0].data

        # get wavemap
        proctab = read_proctab(os.path.join(base, 'kcwir.proc'))
        tab = search_proctab(proctab,
                frame=img_hdu[0], target_type='MARC',
                nearest=True)
        
        ofn = tab['filename'][0].replace('.fits', "_wavemap.fits")
        wavemap_file = os.path.join(redux, ofn)
        wavemap = fits.open(wavemap_file)[0].data

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            ratio = np.nanmedian(img[wavemap > 0] / msci[wavemap > 0])

        crs = img - msci
        crmsk = sigma_clip(crs, sigma = 3, maxiters=1, grow=2).mask.astype(float)
        crmsk_surr = sigma_clip(crs, sigma = 3, maxiters=1, grow=10).mask.astype(float)

        idx = np.where((crmsk < 1e-4) & (crmsk_surr > 1e-4) & (wavemap > 0) )
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            ratio2 = np.nanmedian(img[idx]/msci[idx])
        
        print(ratio, ratio2)

        crmsk_med = crmsk * msci * ratio2
        crmsk_med[crmsk_med < 1e-4] = 0

        primary = fits.PrimaryHDU(crmsk, header=img_hdu[0].header)
        med = fits.ImageHDU(crmsk_med, name='MEDSCI')
        sur = fits.ImageHDU(crmsk_surr, name = 'CRMSK_SUR')
        hdul = fits.HDUList([primary, med, sur])
        hdul.writeto(os.path.join(redux, re.sub('.fits', '_crmsk.fits', f)), overwrite=True)


######################desi094_0_arcs_300##############
kr240211_00080.fits
1.1837709 1.1903323
kr240211_00081.fits
1.0264151 1.0271493
kr240211_00082.fits
1.0 1.0
kr240211_00083.fits
0.99002135 0.9889791
kr240211_00084.fits
0.9881289 0.98635477
kr240211_00085.fits
0.9784615 0.9769484
kr240211_00086.fits
0.9791045 0.9735049
kr240211_00087.fits
0.9695432 0.9679976
######################desi094-sky_0_arcs_300##############
kr240211_00088.fits
1.0161725 1.013204
kr240211_00089.fits
1.0 1.0
######################desi094_90_arcs_300##############
kr240211_00090.fits
1.0449438 1.0439147
kr240211_00091.fits
1.0168675 1.015501
kr240211_00092.fits
0.98621505 0.98426974
kr240211_00093.fits
0.94864285 0.94736844
######################macs0744_0_arcs_300##############
kr240211_00106.fits
1.0 1.0
kr240211_00107.fits
1.0 1.0
kr240211_00108.fits
1.0 1.0
kr240211_00109.fits
1.008596 1.0068234
kr240211_00110.fits
1.0088757 1.0085082
kr240211_00111.fits
1.0 1.0
kr240211_00112.fits
0.9981364 0.995837
kr24

In [74]:
wavemap

array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32)